In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core import Workspace

workspace = Workspace.from_config()

name = "win-compute"

compute_provisioning_config = AmlCompute.provisioning_configuration(
   vm_size="STANDARD_D2_V2",
   min_nodes=0,
   max_nodes=3)

compute_create_payload = AmlCompute._build_create_payload(
   compute_provisioning_config, workspace.location, workspace.subscription_id)
compute_create_payload["properties"]["properties"]["osType"] = 'Windows'
compute_target = ComputeTarget._create_compute_target(
   workspace, name, compute_create_payload, AmlCompute)
compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print(compute_target.get_status().serialize())
print(compute_target.provisioning_errors)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-07-18T17:41:04.010000+00:00', 'errors': None, 'creationTime': '2023-07-18T17:40:58.750594+00:00', 'modifiedTime': '2023-07-18T17:41:05.734309+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 3, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}
None


In [25]:
ws = Workspace.from_config()

In [27]:
import os
from azureml.core import Environment
from azureml.core import Workspace, Experiment, Environment,ScriptRunConfig
from azureml.core.runconfig import RunConfiguration

rayEnv = Environment(name = 'win-env-2', file_path = 'requirements.txt', environment_variables={
        "AZUREML_COMPUTE_USE_COMMON_RUNTIME": "true"
    })
rayEnv.docker.base_image = 'jazmiabonsai1.azurecr.io/windows-env:1'
rayEnv.register(ws)

{
    "assetId": "azureml://locations/eastus/workspaces/1dfb2860-606d-4cb3-9412-40b2f9a3e5f0/environments/win-env-2/versions/2",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "jazmiabonsai1.azurecr.io/windows-env:1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "win-env-2",
    "python": {
        "baseCondaEnviro

In [24]:
# rayEnv.docker.base_image = 'jazmiabonsai1.azurecr.io/windows-env:1'

from azureml.core.runconfig import DockerConfiguration

docker_config = DockerConfiguration(use_docker=True, shm_size='24g')

aml_run_config_ml = RunConfiguration(communicator='OpenMpi')
aml_run_config_ml.target = compute_target
aml_run_config_ml.node_count = 2
aml_run_config_ml.environment = 'CliV2AnonymousEnvironment:72944a7e62f52158fe0aea579958a719'

src = ScriptRunConfig(source_directory= '.',
                    script='main.py',
                    run_config = aml_run_config_ml,
                   )

run = Experiment(ws, 'win-exp').submit(src)

AttributeError: 'str' object has no attribute 'python'